In [1]:
import pandas as pd

In [2]:
from pandas import read_csv
import numpy as np
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
import warnings

warnings.filterwarnings("ignore")


In [3]:
# gene expression data
path = "GSE113513_merged_data_final.csv"
df_train = pd.read_csv(path)

In [4]:
df_train.head()

,geo_accession,11715100_at,11715101_s_at,11715102_x_at,11715103_x_at,11715104_s_at,11715105_at,11715106_x_at,11715107_s_at,11715108_x_at,...,AFFX-r2-TagO-5_at,AFFX-r2-TagQ-3_at,AFFX-r2-TagQ-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at,target
0,GSM3108231,20.550440,38.911007,25.829760,35.282680,20.890873,584.405330,16.210958,19.788046,8.002856,...,8.167891,8.400082,10.856209,341.78528,14.535798,68.162420,6.568994,9.048326,7.072084,non-cancerous colorectal
1,GSM3108232,27.800337,53.513405,33.444916,73.262596,40.296627,8.146114,25.810207,17.198637,10.963716,...,7.324960,7.844143,13.119226,930.18010,24.197203,183.315950,6.111884,10.366261,7.779812,non-cancerous colorectal
2,GSM3108233,16.569454,36.890644,19.229105,48.691338,47.897854,7.184215,15.303843,17.251259,8.699706,...,5.668957,7.923203,9.848433,371.97983,12.700565,80.227210,5.579404,8.286402,7.756152,non-cancerous colorectal
3,GSM3108234,16.016920,43.052690,22.094769,45.083103,74.160830,7.416252,18.476812,16.654469,7.955965,...,7.702706,7.602839,9.054254,267.12810,13.136146,52.165592,5.868696,9.331419,7.682987,non-cancerous colorectal
4,GSM3108235,18.516483,47.595444,26.349796,53.573290,67.538360,9.876798,16.090324,20.565605,10.133324,...,6.485854,7.391753,10.520305,340.54310,11.294785,61.768856,6.452526,8.897550,7.286209,non-cancerous colorectal


In [5]:
df_train.columns

Index(['geo_accession', '11715100_at', '11715101_s_at', '11715102_x_at',
       '11715103_x_at', '11715104_s_at', '11715105_at', '11715106_x_at',
       '11715107_s_at', '11715108_x_at',
       ...
       'AFFX-r2-TagO-5_at', 'AFFX-r2-TagQ-3_at', 'AFFX-r2-TagQ-5_at',
       'AFFX-ThrX-3_at', 'AFFX-ThrX-5_at', 'AFFX-ThrX-M_at', 'AFFX-TrpnX-3_at',
       'AFFX-TrpnX-5_at', 'AFFX-TrpnX-M_at', 'target'],
      dtype='object', length=49397)

In [6]:
df_train = df_train.iloc[:, 1:]

In [7]:
df_train

,11715100_at,11715101_s_at,11715102_x_at,11715103_x_at,11715104_s_at,11715105_at,11715106_x_at,11715107_s_at,11715108_x_at,11715109_at,...,AFFX-r2-TagO-5_at,AFFX-r2-TagQ-3_at,AFFX-r2-TagQ-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at,target
0,20.550440,38.911007,25.829760,35.282680,20.890873,584.405330,16.210958,19.788046,8.002856,14.913201,...,8.167891,8.400082,10.856209,341.78528,14.535798,68.162420,6.568994,9.048326,7.072084,non-cancerous colorectal
1,27.800337,53.513405,33.444916,73.262596,40.296627,8.146114,25.810207,17.198637,10.963716,16.505486,...,7.324960,7.844143,13.119226,930.18010,24.197203,183.315950,6.111884,10.366261,7.779812,non-cancerous colorectal
2,16.569454,36.890644,19.229105,48.691338,47.897854,7.184215,15.303843,17.251259,8.699706,12.608926,...,5.668957,7.923203,9.848433,371.97983,12.700565,80.227210,5.579404,8.286402,7.756152,non-cancerous colorectal
3,16.016920,43.052690,22.094769,45.083103,74.160830,7.416252,18.476812,16.654469,7.955965,13.578931,...,7.702706,7.602839,9.054254,267.12810,13.136146,52.165592,5.868696,9.331419,7.682987,non-cancerous colorectal
4,18.516483,47.595444,26.349796,53.573290,67.538360,9.876798,16.090324,20.565605,10.133324,13.962496,...,6.485854,7.391753,10.520305,340.54310,11.294785,61.768856,6.452526,8.897550,7.286209,non-cancerous colorectal
5,19.599030,48.847095,28.435322,54.268870,70.464090,8.973177,15.787043,16.811987,9.152577,13.365874,...,5.829067,7.770578,8.922758,230.98311,10.751132,44.522137,6.589101,9.924715,6.876703,non-cancerous colorectal
6,14.035622,40.725403,20.063507,44.542330,46.969917,8.313570,17.847345,19.606396,8.700618,19.986029,...,6.471952,8.242336,9.663445,480.04196,11.246759,73.967834,6.723006,9.662200,7.607384,non-cancerous colorectal
7,18.630518,47.139206,26.510470,50.405674,20.381641,7.860251,14.737239,16.854115,9.259340,12.282247,...,6.410641,8.205932,9.516732,238.18990,10.294109,35.933815,6.254692,8.984534,7.602689,non-cancerous colorectal
8,15.249186,41.869810,18.727713,99.521520,30.333733,7.554156,15.826531,16.735561,7.476953,13.489064,...,6.649616,9.060200,9.606091,274.94055,10.702040,57.252970,6.691254,7.797127,7.578697,non-cancerous colorectal
9,16.836504,49.687460,23.065104,65.252560,71.361900,8.200011,13.474164,15.745662,9.259606,13.651482,...,6.867582,7.871133,9.006868,136.75235,8.640280,23.988642,6.525020,7.973787,7.366324,non-cancerous colorectal


In [8]:
import pandas as pd

df_train['target'] = df_train['target'].map({'colorectal carcinoma': 1, 'non-cancerous colorectal': 0})

print(df_train)


    11715100_at  11715101_s_at  11715102_x_at  11715103_x_at  11715104_s_at  \
0     20.550440      38.911007      25.829760      35.282680      20.890873   
1     27.800337      53.513405      33.444916      73.262596      40.296627   
2     16.569454      36.890644      19.229105      48.691338      47.897854   
3     16.016920      43.052690      22.094769      45.083103      74.160830   
4     18.516483      47.595444      26.349796      53.573290      67.538360   
5     19.599030      48.847095      28.435322      54.268870      70.464090   
6     14.035622      40.725403      20.063507      44.542330      46.969917   
7     18.630518      47.139206      26.510470      50.405674      20.381641   
8     15.249186      41.869810      18.727713      99.521520      30.333733   
9     16.836504      49.687460      23.065104      65.252560      71.361900   
10    16.093780      43.784570      26.297377      60.918365      44.427677   
11    17.963362      50.005530      26.506010      3

In [9]:
X_train=df_train.iloc[:,0:-1]
y_train=df_train.iloc[:,-1]

In [10]:
X_train.head()

,11715100_at,11715101_s_at,11715102_x_at,11715103_x_at,11715104_s_at,11715105_at,11715106_x_at,11715107_s_at,11715108_x_at,11715109_at,...,AFFX-r2-TagO-3_at,AFFX-r2-TagO-5_at,AFFX-r2-TagQ-3_at,AFFX-r2-TagQ-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at
0,20.550440,38.911007,25.829760,35.282680,20.890873,584.405330,16.210958,19.788046,8.002856,14.913201,...,9.011358,8.167891,8.400082,10.856209,341.78528,14.535798,68.162420,6.568994,9.048326,7.072084
1,27.800337,53.513405,33.444916,73.262596,40.296627,8.146114,25.810207,17.198637,10.963716,16.505486,...,7.333499,7.324960,7.844143,13.119226,930.18010,24.197203,183.315950,6.111884,10.366261,7.779812
2,16.569454,36.890644,19.229105,48.691338,47.897854,7.184215,15.303843,17.251259,8.699706,12.608926,...,6.461520,5.668957,7.923203,9.848433,371.97983,12.700565,80.227210,5.579404,8.286402,7.756152
3,16.016920,43.052690,22.094769,45.083103,74.160830,7.416252,18.476812,16.654469,7.955965,13.578931,...,7.992805,7.702706,7.602839,9.054254,267.12810,13.136146,52.165592,5.868696,9.331419,7.682987
4,18.516483,47.595444,26.349796,53.573290,67.538360,9.876798,16.090324,20.565605,10.133324,13.962496,...,6.740306,6.485854,7.391753,10.520305,340.54310,11.294785,61.768856,6.452526,8.897550,7.286209


In [11]:
y_train.head()

0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int64

In [12]:
# gene expression data
path = "GSE44076_merged_data.csv"
df_second = pd.read_csv(path)

In [13]:
df_test=df_second

In [14]:
df_test.head()

,geo_accession,sample_type,individual_id,stage,location,gender,age,status,11715100_at,11715101_s_at,...,AFFX-r2-TagO-3_at,AFFX-r2-TagO-5_at,AFFX-r2-TagQ-3_at,AFFX-r2-TagQ-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at
0,GSM1077598,Mucosa,A2119,--,Left,Male,62,Public on Mar 14 2014,3.6599,4.1524,...,1.9475,2.1732,2.3173,2.0892,7.0313,4.8902,5.9995,2.1907,2.1474,2.1102
1,GSM1077599,Mucosa,A2142,--,Left,Female,77,Public on Mar 14 2014,2.5712,4.1879,...,1.9056,1.9503,2.0665,2.0585,7.3201,5.2443,6.2646,1.9371,2.2677,2.3093
2,GSM1077600,Mucosa,B2104,--,Left,Female,78,Public on Mar 14 2014,3.3174,4.1040,...,2.5070,2.4232,2.4662,2.2127,8.0270,5.8140,6.3628,2.1923,2.2233,2.2221
3,GSM1077601,Mucosa,B2127,--,Right,Male,65,Public on Mar 14 2014,3.1835,4.3635,...,2.0228,2.7202,2.4453,2.3259,7.1081,4.7067,5.1356,2.0068,2.2773,2.2656
4,GSM1077602,Mucosa,B2150,--,Right,Female,52,Public on Mar 14 2014,2.9949,4.5227,...,1.9309,2.1284,2.6587,2.4271,7.7353,5.2432,6.1207,2.1847,2.3030,2.1275


In [15]:
# Example list of metadata columns
metadata_columns = ['geo_accession', 'individual_id', 'stage', 'location',
                    'gender', 'age', 'status']

# Drop all metadata columns except 'sample_type'
df_test = df_test.drop(columns=[col for col in metadata_columns if col in df_test.columns])

In [16]:
print(df_test.head())

  sample_type  11715100_at  11715101_s_at  11715102_x_at  11715103_x_at  \
0      Mucosa       3.6599         4.1524         3.3896         3.8929   
1      Mucosa       2.5712         4.1879         3.1228         3.7299   
2      Mucosa       3.3174         4.1040         3.2434         4.1048   
3      Mucosa       3.1835         4.3635         3.1175         3.3426   
4      Mucosa       2.9949         4.5227         3.3627         3.3376   

   11715104_s_at  11715105_at  11715106_x_at  11715107_s_at  11715108_x_at  \
0         7.3799       2.0732         2.8276         2.6345         2.5165   
1         7.7642       2.3151         2.8944         2.4020         2.5873   
2         6.4705       2.1824         2.7407         2.4271         2.5785   
3         7.0046       2.1961         2.4710         2.2932         2.3527   
4         6.9029       2.5087         2.8609         2.4833         2.3683   

   ...  AFFX-r2-TagO-3_at  AFFX-r2-TagO-5_at  AFFX-r2-TagQ-3_at  \
0  ...       

In [17]:
df_test['sample_type'] = df_test['sample_type'].map({'Mucosa': -1, 'Normal': 0, 'Tumor':1})

print(df_test)

     sample_type  11715100_at  11715101_s_at  11715102_x_at  11715103_x_at  \
0             -1       3.6599         4.1524         3.3896         3.8929   
1             -1       2.5712         4.1879         3.1228         3.7299   
2             -1       3.3174         4.1040         3.2434         4.1048   
3             -1       3.1835         4.3635         3.1175         3.3426   
4             -1       2.9949         4.5227         3.3627         3.3376   
..           ...          ...            ...            ...            ...   
241            1       2.7624         3.7898         3.0972         4.3038   
242            1       2.5775         3.7737         2.7807         3.2338   
243            1       3.1441         4.3357         3.4050         3.4187   
244            1       4.8256         5.4895         4.4170         4.6358   
245            1       3.3535         4.4803         3.2669         4.1998   

     11715104_s_at  11715105_at  11715106_x_at  11715107_s_at  

In [18]:
df_test.columns

Index(['sample_type', '11715100_at', '11715101_s_at', '11715102_x_at',
       '11715103_x_at', '11715104_s_at', '11715105_at', '11715106_x_at',
       '11715107_s_at', '11715108_x_at',
       ...
       'AFFX-r2-TagO-3_at', 'AFFX-r2-TagO-5_at', 'AFFX-r2-TagQ-3_at',
       'AFFX-r2-TagQ-5_at', 'AFFX-ThrX-3_at', 'AFFX-ThrX-5_at',
       'AFFX-ThrX-M_at', 'AFFX-TrpnX-3_at', 'AFFX-TrpnX-5_at',
       'AFFX-TrpnX-M_at'],
      dtype='object', length=49387)

In [19]:
df_test.shape

(246, 49387)

In [20]:
target_second='sample_type'

df_test = df_test[[col for col in df_test.columns if col != target_second] + [target_second]]

# Display the modified DataFrame structure
print(df_test.head())

   11715100_at  11715101_s_at  11715102_x_at  11715103_x_at  11715104_s_at  \
0       3.6599         4.1524         3.3896         3.8929         7.3799   
1       2.5712         4.1879         3.1228         3.7299         7.7642   
2       3.3174         4.1040         3.2434         4.1048         6.4705   
3       3.1835         4.3635         3.1175         3.3426         7.0046   
4       2.9949         4.5227         3.3627         3.3376         6.9029   

   11715105_at  11715106_x_at  11715107_s_at  11715108_x_at  11715109_at  ...  \
0       2.0732         2.8276         2.6345         2.5165       2.4795  ...   
1       2.3151         2.8944         2.4020         2.5873       2.2066  ...   
2       2.1824         2.7407         2.4271         2.5785       2.6003  ...   
3       2.1961         2.4710         2.2932         2.3527       2.4741  ...   
4       2.5087         2.8609         2.4833         2.3683       2.3747  ...   

   AFFX-r2-TagO-5_at  AFFX-r2-TagQ-3_at  AFF

In [21]:
X_test=df_test.iloc[:,0:-1]
y_test=df_test.iloc[:,-1]

In [22]:
X_test.head()

,11715100_at,11715101_s_at,11715102_x_at,11715103_x_at,11715104_s_at,11715105_at,11715106_x_at,11715107_s_at,11715108_x_at,11715109_at,...,AFFX-r2-TagO-3_at,AFFX-r2-TagO-5_at,AFFX-r2-TagQ-3_at,AFFX-r2-TagQ-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at
0,3.6599,4.1524,3.3896,3.8929,7.3799,2.0732,2.8276,2.6345,2.5165,2.4795,...,1.9475,2.1732,2.3173,2.0892,7.0313,4.8902,5.9995,2.1907,2.1474,2.1102
1,2.5712,4.1879,3.1228,3.7299,7.7642,2.3151,2.8944,2.4020,2.5873,2.2066,...,1.9056,1.9503,2.0665,2.0585,7.3201,5.2443,6.2646,1.9371,2.2677,2.3093
2,3.3174,4.1040,3.2434,4.1048,6.4705,2.1824,2.7407,2.4271,2.5785,2.6003,...,2.5070,2.4232,2.4662,2.2127,8.0270,5.8140,6.3628,2.1923,2.2233,2.2221
3,3.1835,4.3635,3.1175,3.3426,7.0046,2.1961,2.4710,2.2932,2.3527,2.4741,...,2.0228,2.7202,2.4453,2.3259,7.1081,4.7067,5.1356,2.0068,2.2773,2.2656
4,2.9949,4.5227,3.3627,3.3376,6.9029,2.5087,2.8609,2.4833,2.3683,2.3747,...,1.9309,2.1284,2.6587,2.4271,7.7353,5.2432,6.1207,2.1847,2.3030,2.1275


In [23]:
X_test.shape

(246, 49386)

In [24]:
y_test.head()

0   -1
1   -1
2   -1
3   -1
4   -1
Name: sample_type, dtype: int64

In [25]:
common_cols = set(X_train.columns).intersection(X_test.columns)

In [26]:
common_cols

{'11729631_x_at',
 '11736459_a_at',
 '11737152_at',
 '11721941_x_at',
 '11725039_at',
 '11758384_s_at',
 '11717175_a_at',
 '11722095_a_at',
 '11743251_s_at',
 '11724173_at',
 '11721326_at',
 '11751899_a_at',
 '11739881_a_at',
 '11730880_a_at',
 '11736053_x_at',
 '11727601_a_at',
 '11720332_a_at',
 '11735795_at',
 '11736795_a_at',
 '11750441_a_at',
 '11729630_a_at',
 '11748049_x_at',
 '11754741_x_at',
 '11724341_at',
 '11729539_a_at',
 '11716589_x_at',
 '11726673_s_at',
 '11746234_a_at',
 '11755551_a_at',
 '11725123_a_at',
 '11759232_at',
 '11758867_at',
 '11733506_at',
 '11720153_s_at',
 '11737927_a_at',
 '11730095_at',
 '11744434_a_at',
 '11742238_a_at',
 '11746514_a_at',
 '11734567_a_at',
 '11725695_s_at',
 '11731979_at',
 '11736719_at',
 '11749971_x_at',
 '11729181_at',
 '11724199_x_at',
 '11730435_at',
 '11743380_s_at',
 '11727594_a_at',
 '11733516_at',
 '11759953_at',
 '11763221_x_at',
 '11744335_x_at',
 '11741439_a_at',
 '11725889_at',
 '11762689_x_at',
 '11719818_at',
 '11729644

In [27]:
X_train_common = X_train[list(common_cols)]
X_test_common = X_test[list(common_cols)]

In [28]:
# Split-out validation dataset
validation_size = 0.30
seed = 7
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=validation_size, random_state=seed)

# Model selection
model = LogisticRegression()

# Cross-validation for training accuracy
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
cv_results = cross_val_score(model, X_train_common, y_train, cv=kfold, scoring='accuracy')
print("Training Accuracy: {:.2f}%".format(cv_results.mean() * 100))

# Train the model
model.fit(X_train_common, y_train)


Training Accuracy: 100.00%


LogisticRegression()

In [29]:
# Predictions
predictions = model.predict(X_test_common)

# Calculate testing accuracy
test_accuracy = accuracy_score(y_test, predictions)
print("Testing Accuracy: {:.2f}%".format(test_accuracy * 100))

# Confusion Matrix & Classification Report
print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions))

print("Classification Report:")
print(classification_report(y_test, predictions))

Testing Accuracy: 78.05%
Confusion Matrix:
[[ 0 50  0]
 [ 0 96  2]
 [ 0  2 96]]
Classification Report:
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        50
           0       0.65      0.98      0.78        98
           1       0.98      0.98      0.98        98

    accuracy                           0.78       246
   macro avg       0.54      0.65      0.59       246
weighted avg       0.65      0.78      0.70       246

